In [11]:
import time
import sdl2
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

from brainflow.board_shim import BoardShim, BrainFlowInputParams, LogLevels, BoardIds
from brainflow.data_filter import DataFilter, AggOperations, FilterTypes, WindowOperations, NoiseTypes

In [12]:
board_id = BoardIds.CROWN_BOARD.value
params = BrainFlowInputParams()
params.mac_address = "C0:EE:40:84:DD:56"
params.serial_number = "58a99b0107e64cd40ea5e6607882cbe2"
params.board_id = board_id
params.timeout = 5
BoardShim.enable_dev_board_logger()
board = BoardShim(board_id, params)
board.prepare_session()

[2023-01-30 23:29:46.670] [board_logger] [info] incoming json: {
    "board_id": 23,
    "file": "",
    "file_anc": "",
    "file_aux": "",
    "ip_address": "",
    "ip_address_anc": "",
    "ip_address_aux": "",
    "ip_port": 0,
    "ip_port_anc": 0,
    "ip_port_aux": 0,
    "ip_protocol": 0,
    "mac_address": "C0:EE:40:84:DD:56",
    "master_board": -100,
    "other_info": "",
    "serial_number": "58a99b0107e64cd40ea5e6607882cbe2",
    "serial_port": "",
    "timeout": 5
}
[2023-01-30 23:29:46.670] [board_logger] [trace] Board object created 23
[2023-01-30 23:29:46.670] [board_logger] [debug] Use IP port 9000


In [13]:
sdl2.SDL_Init(sdl2.SDL_INIT_JOYSTICK)
joystick = sdl2.SDL_JoystickOpen(0)

def get_direction():
        sdl2.SDL_PumpEvents()
        # depending on the gamepad this gives you a value between -32768 and +32768
        # or between 0 and 32768
        x = sdl2.SDL_JoystickGetAxis(joystick, 0)
        y = sdl2.SDL_JoystickGetAxis(joystick, 1)
        # x_direction = "right" if joy_x / 32768 > 0.3 else "left" if joy_x / 32768 < -0.1 else "center" # WARNING: A bit offset because of drift, should be 0.2 and -0.2 for the deadzone
        # print(f"X: {joy_x} Y: {joy_y} Direction: {x_direction}                    ", end="\r")

        return x, y

In [8]:
train_test = input("Train or Test? ").lower()

start_time = time.time()
inputs = np.empty((8,0))
outputs = []
BoardShim.disable_board_logger()
board.start_stream()
while(start_time + 60 > time.time()):
    x, y = get_direction()
    direction = 2 if x / 32768 > 0.2 else 0 if x / 32768 < -0.2 else 1
    data = board.get_board_data()[1:9]
    inputs = np.concatenate((inputs, data), axis=1)
    for i in range(data.shape[1]):
        outputs.append(direction)
    text_direction = "right" if x / 32768 > 0.2 else "left" if x / 32768 < -0.2 else "center"
    print(f"X: {x} Y: {y} Direction: {text_direction}                    ", end="\r")
    time.sleep(0.05)

outputs = np.array(outputs)    
board.stop_stream()

In [9]:
now = datetime.now()
dt_string = now.strftime("%d:%m:%Y-%H:%M:%S")

dataset = np.array([inputs, outputs])
np.save(f"data/{train_test}/{dt_string}.npy", dataset)

/tmp/ipykernel_235865/3853576982.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  dataset = np.array([inputs, outputs])


In [32]:
board.release_session()